<a href="https://colab.research.google.com/github/Sicaida/fdc-public/blob/master/PreSumm_BERT_Summerization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Just the absolute minimum required to get this working at all. Most of the work was done in this https://github.com/mingchen62/PreSumm.git fork.

Python 3
Runtime GPU? (Actually GPU doesn't work?)

Also see: https://github.com/nlpyang/PreSumm/issues/11

txt files go in /content/PreSumm/bert_data/cnndm

Just try one file at once, I didn't bother making it really work with multiple files.

results show up in /content/PreSumm/results and also printed in the cell.

Quick sample output with fiction: https://twitter.com/jonathanfly/status/1168377996597764096

In [0]:
from google.colab import drive
from datetime import datetime
drive.mount('/content/drive', force_remount=True)

In [0]:
!git clone https://github.com/mingchen62/PreSumm.git #based on this fork which handles the encoding etc
%cd PreSumm
!pip install pytorch_transformers tensorboardX pyrouge
%cd /content/PreSumm/models
!gdown https://drive.google.com/uc?id=1kKWoV0QCbeIuFt85beQgJ4v0lujaXobJ&export=download #CNN/DM Extractive bertext_cnndm_transformer.pt
!gdown https://drive.google.com/uc?id=1-IKVCtc4Q-BdZpjXc4s70_fRsWnjtYLr&export=download #CNN/DM Abstractive model_step_148000.pt 
!gdown https://drive.google.com/uc?id=1H50fClyTkNprWJNh10HWdGEdDdQIkzsI&export=download #XSUM (One Sentence Summary) model_step_30000.pt   
!unzip /content/PreSumm/models/bertext_cnndm_transformer.zip
!unzip /content/PreSumm/models/bertsumextabs_cnndm_final_model.zip
!unzip /content/PreSumm/models/bertsumextabs_xsum_final_model.zip
!mkdir /content/PreSumm/models/CNN_DailyMail_Extractive
!mkdir /content/PreSumm/models/CNN_DailyMail_Abstractive
!mkdir /content/PreSumm/models/XSUM_OneSentence
!mv /content/PreSumm/models/bertext_cnndm_transformer.pt /content/PreSumm/models/CNN_DailyMail_Extractive
!mv /content/PreSumm/models/model_step_148000.pt /content/PreSumm/models/CNN_DailyMail_Abstractive
!mv /content/PreSumm/models/model_step_30000.pt /content/PreSumm/models/XSUM_OneSentence

Cloning into 'PreSumm'...
remote: Enumerating objects: 154, done.
remote: Total 154 (delta 0), reused 0 (delta 0), pack-reused 154
Receiving objects: 100% (154/154), 12.97 MiB | 17.97 MiB/s, done.
Resolving deltas: 100% (64/64), done.
/content/PreSumm
     |████████████████████████████████| 184kB 3.5MB/s 
     |████████████████████████████████| 225kB 49.7MB/s 
     |████████████████████████████████| 61kB 25.5MB/s 
     |████████████████████████████████| 1.0MB 46.0MB/s 
     |████████████████████████████████| 655kB 42.7MB/s 
     |████████████████████████████████| 860kB 37.0MB/s 
  Created wheel for pyrouge: filename=pyrouge-0.1.3-cp36-none-any.whl size=191613 sha256=9f43f16ea6a02380c567e669a8f3f1aca214f3e2bc434e0ae9d68169ca3c1a91
  Stored in directory: /root/.cache/pip/wheels/75/d3/0c/e5b04e15b6b87c42e980de3931d2686e14d36e045058983599
  Created wheel for regex: filename=regex-2019.8.19-cp36-cp36m-linux_x86_64.whl size=609227 sha256=91d03e720ed4f734a4096e64a4ff7b311c6b1346d6083fcd0b520e

In [0]:
!mkdir /content/PreSumm/bert_data_test/
!mkdir /content/PreSumm/bert_data/cnndm
%cd /content/PreSumm/bert_data/cnndm
#!wget https://pastebin.com/raw/L96bGaXL #first chapter of Game of Thrones
#!wget https://pastebin.com/raw/0CW5z6fp #first chapter Harry Potter

/content/PreSumm/bert_data/cnndm


In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
%%writefile /content/PreSumm/src/summarizer.py
#!/usr/bin/env python
"""
    Main training workflow
"""
from __future__ import division

import argparse
import json
import os
from others.logging import init_logger
from train_abstractive import validate_abs, train_abs, baseline, test_abs, test_text_abs, load_models_abs
from train_extractive import train_ext, validate_ext, test_ext
from prepro import data_builder
import glob, os

model_flags = ['hidden_size', 'ff_size', 'heads', 'emb_size', 'enc_layers', 'enc_hidden_size', 'enc_ff_size',
               'dec_layers', 'dec_hidden_size', 'dec_ff_size', 'encoder', 'ff_actv', 'use_interval']


def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')



def init_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("-task", default='abs', type=str, choices=['ext', 'abs'])
    parser.add_argument("-encoder", default='bert', type=str, choices=['bert', 'baseline'])
    parser.add_argument("-mode", default='test', type=str, choices=['train', 'validate', 'test'])
    parser.add_argument("-bert_data_path", default='../../bert_data_new/cnndm')
    parser.add_argument("-model_path", default='../../models/')
    parser.add_argument("-result_path", default='../../results/cnndm')
    parser.add_argument("-temp_dir", default='../../temp')

    parser.add_argument("-batch_size", default=140, type=int)
    parser.add_argument("-test_batch_size", default=200, type=int)

    parser.add_argument("-max_pos", default=800, type=int)
    parser.add_argument("-use_interval", type=str2bool, nargs='?',const=True,default=True)
    parser.add_argument("-large", type=str2bool, nargs='?',const=True,default=False)
    parser.add_argument("-load_from_extractive", default='', type=str)

    parser.add_argument("-sep_optim", type=str2bool, nargs='?',const=True,default=True)
    parser.add_argument("-lr_bert", default=2e-3, type=float)
    parser.add_argument("-lr_dec", default=2e-3, type=float)
    parser.add_argument("-use_bert_emb", type=str2bool, nargs='?',const=True,default=False)

    parser.add_argument("-share_emb", type=str2bool, nargs='?', const=True, default=False)
    parser.add_argument("-finetune_bert", type=str2bool, nargs='?', const=True, default=True)
    parser.add_argument("-dec_dropout", default=0.2, type=float)
    parser.add_argument("-dec_layers", default=6, type=int)
    parser.add_argument("-dec_hidden_size", default=768, type=int)
    parser.add_argument("-dec_heads", default=8, type=int)
    parser.add_argument("-dec_ff_size", default=2048, type=int)
    parser.add_argument("-enc_hidden_size", default=512, type=int)
    parser.add_argument("-enc_ff_size", default=512, type=int)
    parser.add_argument("-enc_dropout", default=0.2, type=float)
    parser.add_argument("-enc_layers", default=6, type=int)

    # params for EXT
    parser.add_argument("-ext_dropout", default=0.2, type=float)
    parser.add_argument("-ext_layers", default=2, type=int)
    parser.add_argument("-ext_hidden_size", default=768, type=int)
    parser.add_argument("-ext_heads", default=8, type=int)
    parser.add_argument("-ext_ff_size", default=2048, type=int)

    parser.add_argument("-label_smoothing", default=0.1, type=float)
    parser.add_argument("-generator_shard_size", default=32, type=int)
    parser.add_argument("-alpha",  default=0.6, type=float)
    parser.add_argument("-beam_size", default=5, type=int)
    parser.add_argument("-min_length", default=15, type=int)
    parser.add_argument("-max_length", default=150, type=int)
    parser.add_argument("-max_tgt_len", default=140, type=int)

    # params for preprocessing
    parser.add_argument("-shard_size", default=2000, type=int)
    parser.add_argument('-min_src_nsents', default=3, type=int)
    parser.add_argument('-max_src_nsents', default=100, type=int)
    parser.add_argument('-min_src_ntokens_per_sent', default=5, type=int)
    parser.add_argument('-max_src_ntokens_per_sent', default=200, type=int)
    parser.add_argument('-min_tgt_ntokens', default=5, type=int)
    parser.add_argument('-max_tgt_ntokens', default=500, type=int)
    parser.add_argument("-lower", type=str2bool, nargs='?',const=True,default=True)
    parser.add_argument("-use_bert_basic_tokenizer", type=str2bool, nargs='?',const=True,default=False)

 
    parser.add_argument("-param_init", default=0, type=float)
    parser.add_argument("-param_init_glorot", type=str2bool, nargs='?',const=True,default=True)
    parser.add_argument("-optim", default='adam', type=str)
    parser.add_argument("-lr", default=1, type=float)
    parser.add_argument("-beta1", default= 0.9, type=float)
    parser.add_argument("-beta2", default=0.999, type=float)
    parser.add_argument("-warmup_steps", default=8000, type=int)
    parser.add_argument("-warmup_steps_bert", default=8000, type=int)
    parser.add_argument("-warmup_steps_dec", default=8000, type=int)
    parser.add_argument("-max_grad_norm", default=0, type=float)

    parser.add_argument("-save_checkpoint_steps", default=5, type=int)
    parser.add_argument("-accum_count", default=1, type=int)
    parser.add_argument("-report_every", default=1, type=int)
    parser.add_argument("-train_steps", default=1000, type=int)
    parser.add_argument("-recall_eval", type=str2bool, nargs='?',const=True,default=False)


    parser.add_argument('-visible_gpus', default='-1', type=str)
    parser.add_argument('-gpu_ranks', default='0', type=str)
    parser.add_argument('-log_file', default='../../logs/cnndm.log')
    parser.add_argument('-seed', default=666, type=int)

    parser.add_argument("-test_all", type=str2bool, nargs='?',const=True,default=False)
    parser.add_argument("-test_from", default='')
    parser.add_argument("-test_start_from", default=-1, type=int)

    parser.add_argument("-train_from", default='')
    parser.add_argument("-report_rouge", type=str2bool, nargs='?',const=True,default=True)
    parser.add_argument("-block_trigram", type=str2bool, nargs='?', const=True, default=True)

    args = parser.parse_args()
    args.gpu_ranks = [int(i) for i in range(len(args.visible_gpus.split(',')))]
    args.world_size = len(args.gpu_ranks)
    os.environ["CUDA_VISIBLE_DEVICES"] = args.visible_gpus

    init_logger(args.log_file)
    device = "cpu" if args.visible_gpus == '-1' else "cuda"
    device_id = 0 if device == "cuda" else -1

    return args, device_id

if __name__ == '__main__':
    args, device_id = init_args()
    print(args.task, args.mode) 

    cp = args.test_from
    try:
    	step = int(cp.split('.')[-2].split('_')[-1])
    except:
    	step = 0

    predictor = load_models_abs(args, device_id, cp, step)

    all_files = glob.glob(os.path.join('/content/drive/My Drive/summary', '*'))
    #print('Files In Input Dir: ' + str(len(all_files)))
    for i, file in enumerate(all_files):
        with open(file) as f:
            source=f.read().rstrip()

        data_builder.str_format_to_bert(  source, args, '../bert_data_test/cnndm.test.0.bert.pt') 
        args.result_path= '../results/content'+str(i)
        args.bert_data_path= '../bert_data_test/cnndm'
        test_text_abs(args, device_id, cp, step, predictor)
        
        tgt, time_used = test_text_abs(args, device_id, cp, step, predictor)

        # some postprocessing 

        sentences = tgt.split('<q>')
        sentences = [sent.capitalize() for sent in sentences]
        sentences = '. '.join(sentences).rstrip()
        sentences = sentences.replace(' ,', ',')
        sentences = sentences+'.'

        print("summary [{}]".format(sentences))
        print("time used {}".format(time_used))
    
  
#   EXTRACT FROM JSON FILE
#   data2 = {}   
#     with open('/content/drive/My Drive/summary/input.json') as json_file:
#       data2 = json.load(json_file)
#       for p in data2:
#         source = data2[p]['FIELD2']
#         data_builder.str_format_to_bert(  source, args, '../bert_data_test/cnndm.test.0.bert.pt') 
#         args.result_path= '../results/content' #+str(i)
#         args.bert_data_path= '../bert_data_test/cnndm'
#         test_text_abs(args, device_id, cp, step, predictor)

#         tgt, time_used = test_text_abs(args, device_id, cp, step, predictor)

#         # some postprocessing 

#         sentences = tgt.split('<q>')
#         sentences = [sent.capitalize() for sent in sentences]
#         sentences = '. '.join(sentences).rstrip()
#         sentences = sentences.replace(' ,', ',')
#         sentences = sentences+'.'

#         print("summary [{}]".format(sentences))
#         print("time used {}".format(time_used))
        
#         data2[p]['FIELD2'] = sentences

#     with open('/content/drive/My Drive/summary/results.json', 'w') as outfile:
#       json.dump(data2, outfile, indent=4)


# EXTRACT FROM CSV
# import csv
# with open('/content/drive/My Drive/summary/input.csv') as csv_file, open('/content/drive/My Drive/summary/outputExt.csv', 'w') as output:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     writer = csv.writer(output, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
#     for row in csv_reader:      
        
        
#         source = row[1]
#         data_builder.str_format_to_bert(  source, args, '../bert_data_test/cnndm.test.0.bert.pt') 
#         args.result_path= '../results/content' #+str(i)
#         args.bert_data_path= '../bert_data_test/cnndm'
#         test_text_abs(args, device_id, cp, step, predictor)

#         tgt, time_used = test_text_abs(args, device_id, cp, step, predictor)


#         sentences = tgt.split('<q>')
#         sentences = [sent.capitalize() for sent in sentences]
#         sentences = '. '.join(sentences).rstrip()
#         sentences = sentences.replace(' ,', ',')
#         sentences = sentences+'.'

        
#         row[1] = sentences
        
#         new_row = [row[0], row[1]]
#         print(new_row)
#         writer.writerow(new_row)

Overwriting /content/PreSumm/src/summarizer.py


In [0]:
#CNN_DM abstractive
%cd /content/PreSumm/src
!python summarizer.py -task abs -mode test -test_from /content/PreSumm/models/CNN_DailyMail_Abstractive/model_step_148000.pt -batch_size 32 -test_batch_size 500 -bert_data_path .../drive/My%20Drive/summary -log_file ../logs/val_abs_bert_cnndm -report_rouge False  -sep_optim true -use_interval true -visible_gpus 0,1,2 -max_pos 512 -max_src_nsents 100 -max_length 200 -alpha 0.95 -min_length 50 

/content/PreSumm/src
abs test
[2019-09-25 07:15:55,397 INFO] Loading checkpoint from /content/PreSumm/models/CNN_DailyMail_Abstractive/model_step_148000.pt
Namespace(accum_count=1, alpha=0.95, batch_size=32, beam_size=5, bert_data_path='.../drive/My%20Drive/summary', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0, 1, 2], label_smoothing=0.1, large=False, load_from_extractive='', log_file='../logs/val_abs_bert_cnndm', lower=True, lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=500, max_pos=512, max_src_nsents=100, max_src_ntokens_per_sent=200, max_tgt_len=140, max_tgt_ntokens=500, min_length=200, min_src_nsents=3, min_src_ntokens_per_sent=5, min_tgt_ntokens=5, mode='test'

In [0]:
#CNN_DM abstractive short
%cd /content/PreSumm/src
!python summarizer.py -task ext -mode test -test_from /content/PreSumm/models/CNN_DailyMail_Extractive/bertext_cnndm_transformer.pt -batch_size 32 -test_batch_size 500 -bert_data_path .../drive/My%20Drive/summary -log_file ../logs/val_abs_bert_cnndm -report_rouge False  -sep_optim true -use_interval true -visible_gpus 0,1,2 -max_pos 512 -max_src_nsents 100 -max_length 200 -alpha 0.95 -min_length 10 

/content/PreSumm/src
ext test
[2019-09-25 07:21:23,927 INFO] Loading checkpoint from /content/PreSumm/models/CNN_DailyMail_Extractive/bertext_cnndm_transformer.pt
Namespace(accum_count=1, alpha=0.95, batch_size=32, beam_size=5, bert_data_path='.../drive/My%20Drive/summary', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0, 1, 2], label_smoothing=0.1, large=False, load_from_extractive='', log_file='../logs/val_abs_bert_cnndm', lower=True, lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=200, max_pos=512, max_src_nsents=100, max_src_ntokens_per_sent=200, max_tgt_len=140, max_tgt_ntokens=500, min_length=10, min_src_nsents=3, min_src_ntokens_per_sent=5, min_tgt_ntokens=5, mode=

In [0]:
#XSUM Abstractive (One Sntence)
%cd /content/PreSumm/src
!python summarizer.py -task abs -mode test -test_from /content/PreSumm/models/XSUM_OneSentence/model_step_30000.pt -batch_size 32 -test_batch_size 500 -bert_data_path ../bert_data/cnndm -log_file ../logs/val_abs_bert_cnndm -report_rouge False  -sep_optim true -use_interval true -visible_gpus 0,1,2 -max_pos 512 -max_src_nsents 100 -max_length 70 -alpha 0.95 -min_length 10 

/content/PreSumm/src
abs test
[2019-09-25 07:19:15,104 INFO] Loading checkpoint from /content/PreSumm/models/XSUM_OneSentence/model_step_30000.pt
Namespace(accum_count=1, alpha=0.95, batch_size=32, beam_size=5, bert_data_path='../bert_data/cnndm', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0, 1, 2], label_smoothing=0.1, large=False, load_from_extractive='', log_file='../logs/val_abs_bert_cnndm', lower=True, lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=200, max_pos=512, max_src_nsents=100, max_src_ntokens_per_sent=200, max_tgt_len=140, max_tgt_ntokens=500, min_length=100, min_src_nsents=3, min_src_ntokens_per_sent=5, min_tgt_ntokens=5, mode='test', model_path='../../

In [0]:
zip_path = '/content/run_{:%Y%m%d_%H%M%S}.zip'.format(datetime.utcnow())
!zip -r zip_path /content/PreSumm/results
#from google.colab import files
#files.download("/content/results.zip")